<a href="https://colab.research.google.com/github/liminal-mradul/monte-carlo/blob/main/Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install gmpy2 numpy
import math
import random
from decimal import Decimal, getcontext
from concurrent.futures import ProcessPoolExecutor
import gmpy2
from gmpy2 import mpz, gcd

# Set precision to 1000+ digits
getcontext().prec = 1010

def is_coprime(a, b):
    """Check if two numbers are coprime using gmpy2 for speed."""
    return gcd(mpz(a), mpz(b)) == 1

def worker(samples):
    """Worker function that can be pickled (defined at module level)."""
    random.seed()  # Ensure each worker has a unique seed
    coprime_count = 0
    for _ in range(samples):
        a = random.getrandbits(64)
        b = random.getrandbits(64)
        if is_coprime(a, b):
            coprime_count += 1
    return coprime_count

def monte_carlo_pi_parallel(total_samples, num_workers=8):
    """Parallel Monte Carlo coprimality sampling (Colab-compatible)."""
    samples_per_worker = total_samples // num_workers
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(worker, [samples_per_worker] * num_workers))

    total_coprime = sum(results)
    p = Decimal(total_coprime) / Decimal(total_samples)
    return (Decimal(6) / p).sqrt()

def verify_bbp_pi(digits):
    """BBP formula for verifying π digits (deterministic)."""
    pi = Decimal(0)
    for k in range(digits + 10):  # Extra terms for accuracy
        term = (Decimal(4)/(8*k+1) - Decimal(2)/(8*k+4) -
               Decimal(1)/(8*k+5) - Decimal(1)/(8*k+6)) / Decimal(16)**k
        pi += term
    return pi

if __name__ == "__main__":
    # Step 1: Monte Carlo approximation (adjust samples for speed/precision)
    print("Running Monte Carlo approximation...")
    approx_pi = monte_carlo_pi_parallel(total_samples=10**7)  # Start with 10M samples for testing
    print(f"Monte Carlo Approx (first 50 digits): {str(approx_pi)[:50]}...")

    # Step 2: Verify with BBP (deterministic)
    print("\nComputing reference π using BBP formula...")
    true_pi = verify_bbp_pi(1000)
    print(f"Verified π (BBP, first 50 digits): {str(true_pi)[:50]}...")

Running Monte Carlo approximation...
Monte Carlo Approx (first 50 digits): 3.141450297569763701651428474423653890309437628402...

Computing reference π using BBP formula...
Verified π (BBP, first 50 digits): 3.141592653589793238462643383279502884197169399375...
